# Test competencia Métodos Base de datos Conteo

Javier Arturo Rozo Alzate - jarozoa@eafit.edu.co  
Alejandro Palacio Vasquez - apalac19@eafit.edu.co  
Liceth Cristina Mosquera Galvis - lcmosquerg@eafit.edu.co  
Cristian David Muñoz Mora - cdmunozm@eafit.edu.co  
Programa: Metodos Estadısticos Avanzados  
Docente:Andres Ramirez Hassan - aramir21@eafit.edu.co  
12 de octubre de 2019<br>

# Análisis Exploratorios de datos

Bibliotecas

In [ ]:
# Pandas para cargar los datos y manipularlos
import pandas as pd


# Funciones numéricas

import xgboost as xgb
from xgboost import XGBRegressor
import numpy as np
from numpy import sort
import collections
import math
import operator
from scipy import stats
from math import ceil
import time
import random as rnd
import statsmodels
import pandas_profiling
 
# Graficar
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
%matplotlib inline

# Modelar
from collections import OrderedDict
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score,train_test_split
from sklearn.metrics import make_scorer,r2_score,mean_squared_error,explained_variance_score
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

from sklearn.preprocessing import StandardScaler,minmax_scale
from sklearn.feature_selection import SelectFromModel
from statsmodels.formula.api import poisson, negativebinomial,glm



import warnings
warnings.filterwarnings("ignore")

# Carga de datos

In [ ]:
#cargar los datos y escoger la variable dependiente
datos = pd.read_csv('datacountstudents.csv')
datos = datos.iloc[:,1:]
Xd = datos.drop(['yC'], axis=1)
yd = datos['yC'].values

In [ ]:
#cargar el test y escoger la variable dependiente
test = pd.read_csv('testcountstudents.csv')
test = test.iloc[:,1:]
Xt = test.drop(['yC'], axis=1)
yt = test['yC'].values

In [ ]:
# Nombres columnas continuas
train_C = Xd.select_dtypes(exclude=['int64'])
Columns_C = Xd.columns[(Xd.dtypes.values == np.dtype('float64'))]

# Estandarizar solo continuas en train
Xd_sc = Xd.copy()
col_names = Columns_C[0:13]
features_tr = Xd_sc[col_names]
scaler = StandardScaler().fit(features_tr.values)
features_tr = scaler.transform(features_tr.values)
Xd_sc[col_names] = features_tr

# Estandarizar solo continuas en test
Xt_sc = Xt.copy()
features_ts = Xt_sc[col_names]
features_ts = scaler.transform(features_ts.values)
Xt_sc[col_names] = features_ts

# Dataframes
X_train_sc = pd.concat([Xd_sc], axis=1, sort=False)
X_test_sc = pd.concat([Xt_sc], axis=1, sort=False)
#X_train_sc.head()

# Modelo

Divido los datos para entrenamiento y testeo

Se construye el modelo con las variables escogidas

In [ ]:
selection_model = XGBRegressor(colsample_bytree=0.6, eta=0.3,learning_rate= 0.1,max_depth=3,min_child_weight=1.5
                                   , n_estimators=1000,reg_alpha=1e-05,reg_lambda= 0.45, subsample=0.6,seed=42
                                   , base_score=0.7 , booster='gbtree' , colsample_bylevel=0.6 , gamma=0.1
                                   , max_delta_step=3 , missing=None , n_jobs=1, nthread=None 
                                   , objective='reg:squarederror', random_state=0, scale_pos_weight=1 , silent=True)

In [ ]:
columns=['x25','x3','x23','x8']# 'x26','x7','x3','x8'#columns=['x25','x3','x23','x8'] #'x8','x23','x11','x2'
# Dataframe y test
Xd_m=pd.DataFrame(Xd_sc,columns=columns)
Xt_m=pd.DataFrame(Xt_sc,columns=columns)
y_t = pd.DataFrame(yt,columns=['yt'])

# Ajustar el Modelo
selection_model.fit(Xd_m,yd)

# Hacer modelo de predicción
preds = selection_model.predict(Xt_m)
preds=np.round(preds)

# DIscretizar la predicción

pred= pd.DataFrame(preds,columns=['pred'])

#anexar columna evaluando si el modelo predice los ceros como ceros
Result1= pd.concat([y_t,pred], axis=1, sort=False)

sum=0
# columnas convirtiendo los mayores a cero como unos tanto en test como en la predicción
Result1['Mayor0_test'] = (Result1['pred']>0).astype(int)
Result1['Mayor0_real'] = (Result1['yt']>0).astype(int)
Result1['Prueba']= [0]*len(Result1['Mayor0_real'])
sum=0
for i in range(len(Result1['Mayor0_real'])):
    if Result1.Mayor0_real[i]==Result1.Mayor0_test[i]:
        Result1['Prueba'][i]=1
        sum=sum+1
    else: Result1['Prueba'] [i]=0

In [ ]:
#print("Explained_variance_score: ",explained_variance_score(yt,pred))
print("Test 0 y >0: ",sum/len(Result1['Mayor0_real']))
print("MSE_test:",(mean_squared_error(yt, pred)))

%matplotlib notebook
confusion_matrix = pd.crosstab(Result1['Mayor0_test'],Result1['Mayor0_real'], rownames=['Actual'], colnames=['Predicted'])
print (confusion_matrix)

#sns.heatmap(confusion_matrix)
sns.set(font_scale=2)
sns.heatmap(confusion_matrix, cmap="Blues", annot=True,annot_kws={"size": 20})# font size

In [ ]:
for i in range (len(columns)):
    v=vif(np.matrix(Xd_m[0:]),i)
    print("Variance inflation factor for {}: {}".format(Xd_m.columns[i],round(v,2)))

In [ ]:
#Result1 = Result1.to_csv ('Desktop\Result1.csv', index = None, header=True)
#Result1

In [ ]:
%matplotlib notebook
plt.plot(range(len(y_t)), y_t, 'r*-', range(len(y_t)), pred, 'bo-')
plt.title('Y_test Vs Y_predit')
plt.legend(['Real Values', 'Fitted Values'])
plt.show()

Validación Cruzada

In [ ]:
#cargando datos en matriz de XGB
columns=['x25','x3','x23','x8']
Xt_m==pd.DataFrame(Xt_m,columns=columns)
data_dmatrix = xgb.DMatrix(data=Xt_m,label=yt )
#obtener los mejores parametros de la iteración
params ={'colsample_bytree': 0.6, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 1.5, 
         'n_estimators': 100, 'reg_alpha': 0.75, 'reg_lambda': 0.01, 'subsample': 0.6}
#entrenando el modelo
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=15)
# Validación cruzada del modelo
cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)
cv_results.head()

In [ ]:
print((cv_results["test-rmse-mean"]).tail(5))